# The UVM Test

### The run\_phase Method

*Note: The code below needs to be run in a freshly started Jupyter notebook kernal. This means after you run it once, you must restart the kernal by pressing the circular arrow restart button above or selecting Kernal -> Restart in the menus to run it again.*

In [4]:
from pyuvm import *
from tb_pkg import *
import random
import time

class AluTest(uvm_test):
    @staticmethod
    def alu_op(A, B, op, is_dut = False):
        """Python model of the TinyALU"""
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        if is_dut:
            time.sleep(0.1)
            if is_dut and (random.randint(0,3) == 0):
                result = result + 1
        return result

    def run_phase(self):
        self.raise_objection()  ## You MUST raise an objection
        cvg = set() #functional coverage
        for _ in range(5):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            cvg.add(op)
            predicted_result = self.alu_op(A, B, op)
            actual_result = self.alu_op(A, B, op, is_dut=True)
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} = {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")
        if len(set(Ops) - cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-cvg}")
        self.drop_objection()  ## drop the objection to end
uvm_root().run_test("AluTest")

INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: ae XOR 5f = 00f1
INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: 1a MUL 2b = 045e
INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: c5 ADD 20 = 00e5
INFO: <ipython-input-4-639ec6314069>(36)[uvm_test_top]: PASSED: df AND f7 = 00d7
ERROR: <ipython-input-4-639ec6314069>(38)[uvm_test_top]: FAILED: 92 ADD 94 = 0127 expected 0126
